# Strategy analysis example

Debugging a strategy can be time-consuming. Freqtrade offers helper functions to visualize raw data.
The following assumes you work with SampleStrategy, data for 5m timeframe from Binance and have downloaded them into the data directory in the default location.
Please follow the [documentation](https://www.freqtrade.io/en/stable/data-download/) for more details.

### Configure Freqtrade environment

In [5]:
from freqtrade.configuration import Configuration
from freqtrade.data.history import load_pair_history
from freqtrade.enums import CandleType
from pathlib import Path
import talib.abstract as ta

cwd = Path().cwd().parent

config = Configuration.from_files([cwd / "config.json"])

config["strategy"] = "VWAPRL"
config["freqaimodel"] = "CatboostClassifier"
config["timerange"] = "20210216-20230212"

pair="ETH/USDT:USDT"

datadir = cwd / rf"data/{ config['exchange']['name'] }"

candles = load_pair_history(pair,
                            config["timeframe"],
                            datadir,
                            data_format = "hdf5",  # Make sure to update this to your data
                            candle_type=CandleType.FUTURES)

candles.tail()


,date,open,high,low,close,volume
74858,2023-02-19 18:30:00+00:00,1686.0,1687.95,1682.25,1683.00,351600.0
74859,2023-02-19 18:45:00+00:00,1683.0,1686.50,1682.65,1685.50,251952.0
74860,2023-02-19 19:00:00+00:00,1685.5,1689.15,1685.10,1688.30,274280.0
74861,2023-02-19 19:15:00+00:00,1688.3,1690.10,1686.45,1688.95,142148.0
74862,2023-02-19 19:30:00+00:00,1688.8,1690.15,1685.50,1686.80,189809.0


## Load and run strategy
* Rerun each time the strategy file is changed

In [6]:
from freqtrade.resolvers import StrategyResolver
from freqtrade.data.dataprovider import DataProvider

strategy = StrategyResolver.load_strategy(config)

strategy.load_freqAI_model()

strategy.dp = DataProvider(config, None, None)

df = strategy.analyze_ticker(candles, {'pair': pair})

df.tail()

AttributeError: 'CatboostClassifier' object has no attribute 'model'

### Display the trade details

In [ ]:
# Report results
print(f"Generated {df['enter_long'].sum()} entry signals")
data = df.set_index('date', drop=False)
data.tail()

## Load Existing Backtest Data 

In [ ]:
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats

backtest_dir = config["user_data_dir"] / "backtest_results"
# backtest_dir = backtest_dir / "backtest-result-2020-07-01_20-04-22.json"

# You can get the full backtest statistics by using the following command.
# This contains all information used to generate the backtest result.
stats = load_backtest_stats(backtest_dir)

# All statistics are available per strategy, so if `--strategy-list` was used during backtest, this will be reflected here as well.
# Example usages:
print(stats['strategy'][config["strategy"]]['results_per_pair'])
# Get pairlist used for this backtest
print(stats['strategy'][config["strategy"]]['pairlist'])
# Get market change (average change of all pairs from start to end of the backtest period)
print(stats['strategy'][config["strategy"]]['market_change'])
# Maximum drawdown ()
print(stats['strategy'][config["strategy"]]['max_drawdown'])
# Maximum drawdown start and end
print(stats['strategy'][config["strategy"]]['drawdown_start'])
print(stats['strategy'][config["strategy"]]['drawdown_end'])


# Get strategy comparison (only relevant if multiple strategies were compared)
print(stats['strategy_comparison'])


In [ ]:
# Load backtested trades as dataframe
trades = load_backtest_data(backtest_dir)

# Show value-counts per pair
trades.groupby("pair")["exit_reason"].value_counts()

## Plotting daily profit / equity line

In [ ]:
# Plotting equity line (starting with 0 on day 1 and adding daily profit for each backtested day)

from freqtrade.configuration import Configuration
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats
import plotly.express as px
import pandas as pd

# strategy = 'SampleStrategy'
# config = Configuration.from_files(["user_data/config.json"])
# backtest_dir = config["user_data_dir"] / "backtest_results"

stats = load_backtest_stats(backtest_dir)
strategy_stats = stats['strategy'][config["strategy"]]

dates = []
profits = []
for date_profit in strategy_stats['daily_profit']:
    dates.append(date_profit[0])
    profits.append(date_profit[1])

equity = 0
equity_daily = []
for daily_profit in profits:
    equity_daily.append(equity)
    equity += float(daily_profit)


df = pd.DataFrame({'dates': dates,'equity_daily': equity_daily})

fig = px.line(df, x="dates", y="equity_daily")
fig.show()


### Load live trading results

In [ ]:
from freqtrade.data.btanalysis import load_trades_from_db

# Fetch trades from database
trades = load_trades_from_db("sqlite:///tradesv3.sqlite")

# Display results
trades.groupby("pair")["exit_reason"].value_counts()

## Analyze the loaded trades for trade parallelism
This can be useful to find the best `max_open_trades` parameter, when used with backtesting in conjunction with `--disable-max-market-positions`.

In [ ]:
from freqtrade.data.btanalysis import analyze_trade_parallelism

# Analyze the above
parallel_trades = analyze_trade_parallelism(trades, '5m')

parallel_trades.plot()

## Plot results

Freqtrade offers interactive plotting capabilities based on plotly.

In [ ]:
from freqtrade.plot.plotting import  generate_candlestick_graph
# Limit graph period to keep plotly quick and reactive

# Filter trades to one pair
trades_red = trades.loc[trades['pair'] == pair]

data_red = data['2019-06-01':'2019-06-10']
# Generate candlestick graph
graph = generate_candlestick_graph(pair=pair,
                                   data=data_red,
                                   trades=trades_red,
                                   indicators1=['sma20', 'ema50', 'ema55'],
                                   indicators2=['rsi', 'macd', 'macdsignal', 'macdhist']
                                  )




In [ ]:
# Show graph inline
# graph.show()

# Render graph in a separate window
graph.show(renderer="browser")


## Plot average profit per trade as distribution graph

In [ ]:
import plotly.figure_factory as ff

hist_data = [trades.profit_ratio]
group_labels = ['profit_ratio']  # name of the dataset

fig = ff.create_distplot(hist_data, group_labels, bin_size=0.01)
fig.show()
